# Graphs for complex code

> In this post, you'll go through some of the scenarios from the lesson `Creating graphs for complex code`. This is the summary of lecture "Custom and Distributed Training with Tensorflow" from DeepLearning.AI.

- toc: true 
- badges: true
- comments: true
- author: Chanseok Kang
- categories: [Python, Coursera, Tensorflow, DeepLearining.AI]
- image: 

## Packages

In [2]:
import tensorflow as tf

As you saw in the lectures, seemingly simple functions can sometimes be difficult to write in graph mode. Fortunately, Autograph generates this complex graph code for us.

- Here is a function that does some multiplication and additon.

In [3]:
a = tf.Variable(1.0)
b = tf.Variable(2.0)

In [4]:
@tf.function
def f(x, y):
    a.assign(y * b)
    b.assign_add(x * a)
    return a + b

In [5]:
print(f(1.0, 2.0))

tf.Tensor(10.0, shape=(), dtype=float32)


In [6]:
print(tf.autograph.to_code(f.python_function))

def tf__f(x, y):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        ag__.converted_call(ag__.ld(a).assign, ((ag__.ld(y) * ag__.ld(b)),), None, fscope)
        ag__.converted_call(ag__.ld(b).assign_add, ((ag__.ld(x) * ag__.ld(a)),), None, fscope)
        try:
            do_return = True
            retval_ = (ag__.ld(a) + ag__.ld(b))
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



- Here is a function that checks if the sign of a number is positive or not.

In [7]:
@tf.function
def sign(x):
    if x > 0:
        return "Positive"
    else:
        return "Nagative or Zero"

In [8]:
print("Sign = {}".format(sign(tf.constant(2))))

Sign = b'Positive'


In [9]:
print("Sign = {}".format(sign(tf.constant(-2))))

Sign = b'Nagative or Zero'


In [10]:
print(tf.autograph.to_code(sign.python_function))

def tf__sign(x):
    with ag__.FunctionScope('sign', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal retval_, do_return
            (do_return, retval_) = vars_

        def if_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = 'Positive'
            except:
                do_return = False
                raise

        def else_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = 'Nagative or Zero'
            except:
                do_return = False
                raise
        ag__.if_stmt((ag__.ld(x) > 0), if_body, else_body, get_state, set_state, ('do_return', '

- Here is another function that includes a while loop.

In [11]:
@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

In [12]:
print(tf.autograph.to_code(f.python_function))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body():
            nonlocal x
            ag__.converted_call(ag__.ld(tf).print, (ag__.ld(x),), None, fscope)
            x = ag__.converted_call(ag__.ld(tf).tanh, (ag__.ld(x),), None, fscope)

        def loop_test():
            return (ag__.converted_call(ag__.ld(tf).reduce_sum, (ag__.ld(x),), None, fscope) > 1)
        ag__.while_stmt(loop_test, loop_body, get_state, set_state, ('x',), {})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



- Here is a function that uses a for loop and an if statement.

In [13]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s

In [14]:
print(tf.autograph.to_code(sum_even.python_function))

def tf__sum_even(items):
    with ag__.FunctionScope('sum_even', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        s = 0

        def get_state_2():
            return (s,)

        def set_state_2(vars_):
            nonlocal s
            (s,) = vars_

        def loop_body(itr):
            nonlocal s
            c = itr
            continue_ = False

            def get_state():
                return (continue_,)

            def set_state(vars_):
                nonlocal continue_
                (continue_,) = vars_

            def if_body():
                nonlocal continue_
                continue_ = True

            def else_body():
                nonlocal continue_
                pass
            ag__.if_stmt(((ag__.ld(c) % 2) > 0), if_body, else_body, get_state, set_state, ('continue_',), 1)

           

## Print statements

Tracing also behaves differently in graph mode. First, here is a function (not decorated with `@tf.function` yet) that prints the value of the input parameter.  `f(2)` is called in a for loop 5 times, and then `f(3)` is called.

In [15]:
def f(x):
    print("Traced with", x)

In [16]:
for i in range(5):
    f(2)

Traced with 2
Traced with 2
Traced with 2
Traced with 2
Traced with 2


In [17]:
f(3)

Traced with 3


If you were to decorate this function with `@tf.function` and run it, notice that the print statement only appears once for `f(2)` even though it is called in a loop.

In [18]:
@tf.function
def f(x):
    print("Traced with", x)

In [19]:
for i in range(5):
    f(2)

Traced with 2


In [20]:
f(3)

Traced with 3


Now compare `print` to `tf.print`.
- `tf.print` is graph aware and will run as expected in loops. 

Try running the same code where `tf.print()` is added in addition to the regular `print`.
- Note how `tf.print` behaves compared to `print` in graph mode.

In [21]:
@tf.function
def f(x):
    print("Traced with", x)
    tf.print("Executed with", x)

In [22]:
for i in range(5):
    f(2)

Traced with 2
Executed with 2
Executed with 2
Executed with 2
Executed with 2
Executed with 2


In [23]:
f(3)

Traced with 3
Executed with 3


## Avoid defining variables inside the function

This function (not decorated yet) defines a tensor `v` and adds the input `x` to it.  

Here, it runs fine.

In [24]:
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return v

In [25]:
f(1)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>

Now if you decorate the function with `@tf.function`.

The cell below will throw an error because `tf.Variable` is defined within the function. The graph mode function should only contain operations.

In [27]:
@tf.function
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return v

In [28]:
f(1)

ValueError: in user code:

    <ipython-input-27-81cbed3b62ed>:3 f  *
        v = tf.Variable(1.0)
    C:\Users\kcsgo\anaconda3\lib\site-packages\tensorflow\python\ops\variables.py:262 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    C:\Users\kcsgo\anaconda3\lib\site-packages\tensorflow\python\ops\variables.py:256 _variable_v2_call
        shape=shape)
    C:\Users\kcsgo\anaconda3\lib\site-packages\tensorflow\python\ops\variables.py:67 getter
        return captured_getter(captured_previous, **kwargs)
    C:\Users\kcsgo\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py:769 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


To get around the error above, simply move `v = tf.Variable(1.0)` to the top of the cell before the `@tf.function` decorator.

In [29]:
# define the variable outside of the decorated function
v = tf.Variable(1.0)

@tf.function
def f(x):
    return v.assign_add(x)

In [30]:
f(5)

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>